In [1]:
import astropy.units as u
import astropy
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.coordinates import get_body_barycentric, get_body, get_moon
from astropy import coordinates


# enter your location here.
# Use this website to know your gps location https://www.gps-coordinates.net/
home = EarthLocation(lat=43.60583786729394*u.deg, lon=1.4486146618587554*u.deg, height=143*u.m)

# visiblity section here.
azSegmentsHome = [ [coordinates.Latitude(35*u.deg),coordinates.Latitude(90*u.deg),
                    coordinates.Longitude(195*u.deg),coordinates.Longitude(359.9999*u.deg)],
                  
             [coordinates.Latitude(40*u.deg),coordinates.Latitude(90*u.deg),
              coordinates.Longitude(0*u.deg),coordinates.Longitude(10*u.deg)],
            ]

STARTDATE = "2019-09-29" 
ENDDATE =  "2019-11-22"
TIMEZONE = "Europe/Paris"


nameTracked = [
    ("M1","deep"),
    ("M8","deep"),
    ("M17","deep"),
    ("M20","deep"),
    ("M27","deep"),
    ('M33','deep'),
    ("M42","deep"),
    ("M43","deep"),
    ("M57","deep"),
    ("M76","deep"),
    ("M78","deep"),
    ("M97","deep"),
    ("polaris","star"),
]

bodyTracked = [
    
    ("moon" , "next"),
    ("neptune" , "planet"),
    ("jupiter" , "planet"),
    ("saturn" , "planet"),
    ("mars" , "planet"),
    ("venus" , "planet"),
]

In [2]:
%matplotlib inline

In [3]:
import operator
import functools 
import pandas as pd
import numpy as np


import time

from ics import Calendar, Event
from multiprocessing.pool import ThreadPool

tp = ThreadPool(40)  #creating a thread pool for future use

In [4]:

class ViewWindow():
    def __init__(self,azSegments):
        self.azSegments = azSegments
    
    def isLongitudeVisible(self,lookAt,alt):
        ev = [ (lookAt.is_within_bounds(mi,ma) and alt>miAlt and alt<maAlt) for miAlt,maAlt,mi,ma in self.azSegments]
        return functools.reduce( operator.or_ , ev)

    def isAltVisible(self,lookAt):
        return lookAt>self.minAlt

    def isVisible(self,az,alt):
        return self.isLongitudeVisible(az,alt)



In [5]:

def nightsDef(dateStartStr,dateEndStr,tz, loc ):
    
    daterange= pd.date_range('%s 12:00:00'%dateStartStr,end=dateEndStr,freq="10T",tz=tz)
    noSunDates = []

    nightDate=daterange[0].date()
    for dt in daterange:
       
        ti = Time(dt)
        
        ob = get_body("sun",ti)
        altaz = ob.transform_to(AltAz(obstime=ti,location=loc))

        if altaz.alt.degree < -10:
            noSunDates.append((ti,nightDate))
        else : 
            nightDate = dt.date()
    return noSunDates


In [6]:

def getInfo(ti,nightDate,nameTracked,bodyTracked,tlsVS):
    altazed = []
    
    
    objects = []
    for key,typev in nameTracked:
        ob = SkyCoord.from_name(key)
        objects.append((key,typev,ob))
    for key,typev in bodyTracked:
        
        ob = get_body(key,ti)
        objects.append((key,typev,ob))

    
    for key,typev,ob in objects:
        altaz = ob.transform_to(AltAz(obstime=ti,location=loc))
        
        
        isVisible = tlsVS.isVisible(altaz.az,altaz.alt)
        altazed.append((ti,nightDate,key,typev,altaz.alt.degree,altaz.az.degree,isVisible))
    return altazed


In [7]:
azSegments = azSegmentsHome
loc=home
tlsVS = ViewWindow(azSegments)


noSunDates = nightsDef(STARTDATE ,ENDDATE,TIMEZONE,loc )


len(noSunDates)

3742

In [8]:

nw = time.time()
buff = tp.map( lambda x:getInfo(x[0],x[1],nameTracked,bodyTracked,tlsVS), noSunDates)
altazed=[]
for k in buff:
    altazed.extend(k)
print(time.time()-nw)

1149.4238595962524


In [9]:
df = pd.DataFrame(altazed,columns=("dt","night","name","typev","alt","az","isVisible"))

df["dt"] = df["dt"].apply(lambda x:x.to_datetime())

In [10]:
# saving calendar full dataframe for later reference
df.to_pickle("calendar_%s_%s.pickle"%(STARTDATE,ENDDATE))

# Treating data now.

In [11]:
df = pd.read_pickle("calendar.pickle")




In [12]:
df = pd.DataFrame(altazed,columns=("dt","night","name","typev","alt","az","isVisible"))

df["dt"] = df["dt"].apply(lambda x:x.to_datetime())


visibles = df[df["isVisible"] == True][["dt","night","name","typev","alt","az"]]

In [13]:
gb = visibles.groupby(["night","name"])
cal = gb["dt"].agg([np.min, np.max])

In [14]:
cal["dur"] = cal["amax"]-cal["amin"]
cal["dur"] = cal["dur"].apply(lambda x:x.total_seconds())
cal = cal[cal["dur"]>60*60]

In [15]:
calri = cal.reset_index()

In [16]:
dmin = calri.join(df.set_index(["dt","name"]),on=["amin","name"],rsuffix="_start")

In [17]:
cal = dmin.join(df.set_index(["dt","name"]),on=["amax","name"],rsuffix="_end")

In [18]:
cal= cal.rename(columns={"alt":"alt_start","az":"az_start"})

In [19]:
cal

,night,name,amin,amax,dur,night_start,typev,alt_start,az_start,isVisible,night_end,typev_end,alt_end,az_end,isVisible_end
0,2019-09-29,M27,2019-09-29 19:50:00,2019-09-29 23:30:00,13200.0,2019-09-29,deep,68.360031,198.042134,True,2019-09-29,deep,35.290558,268.907621,True
1,2019-09-29,M33,2019-09-30 01:20:00,2019-09-30 04:50:00,12600.0,2019-09-29,deep,76.244026,203.021159,True,2019-09-29,deep,42.419175,274.959081,True
2,2019-09-29,M57,2019-09-29 18:40:00,2019-09-29 22:50:00,15000.0,2019-09-29,deep,78.425924,206.370823,True,2019-09-29,deep,36.579648,283.373273,True
3,2019-09-29,M76,2019-09-30 01:00:00,2019-09-30 04:50:00,13800.0,2019-09-29,deep,81.907676,4.219381,True,2019-09-29,deep,51.960781,302.735776,True
4,2019-09-29,polaris,2019-09-29 18:40:00,2019-09-30 04:50:00,36600.0,2019-09-29,star,43.327570,0.820918,True,2019-09-29,star,44.121561,359.433994,True
5,2019-09-30,M27,2019-09-30 19:50:00,2019-09-30 23:20:00,12600.0,2019-09-30,deep,68.124984,200.410666,True,2019-09-30,deep,36.391571,267.835881,True
6,2019-09-30,M33,2019-10-01 01:10:00,2019-10-01 05:00:00,13800.0,2019-09-30,deep,76.627516,197.690262,True,2019-09-30,deep,39.904586,277.135285,True
7,2019-09-30,M57,2019-09-30 18:30:00,2019-09-30 22:50:00,15600.0,2019-09-30,deep,78.862519,200.283905,True,2019-09-30,deep,35.886058,283.929701,True
8,2019-09-30,M76,2019-10-01 01:00:00,2019-10-01 05:00:00,14400.0,2019-09-30,deep,81.933116,359.873586,True,2019-09-30,deep,49.841938,303.445751,True
9,2019-09-30,polaris,2019-09-30 18:30:00,2019-10-01 05:00:00,37800.0,2019-09-30,star,43.311859,0.810238,True,2019-09-30,star,44.095626,359.391578,True


In [20]:
cgb = cal.groupby(["night","name"])

In [21]:
cgb.groups

{(Timestamp('2019-09-29 00:00:00'), 'M27'): Int64Index([0], dtype='int64'),
 (Timestamp('2019-09-29 00:00:00'), 'M33'): Int64Index([1], dtype='int64'),
 (Timestamp('2019-09-29 00:00:00'), 'M57'): Int64Index([2], dtype='int64'),
 (Timestamp('2019-09-29 00:00:00'), 'M76'): Int64Index([3], dtype='int64'),
 (Timestamp('2019-09-29 00:00:00'), 'polaris'): Int64Index([4], dtype='int64'),
 (Timestamp('2019-09-30 00:00:00'), 'M27'): Int64Index([5], dtype='int64'),
 (Timestamp('2019-09-30 00:00:00'), 'M33'): Int64Index([6], dtype='int64'),
 (Timestamp('2019-09-30 00:00:00'), 'M57'): Int64Index([7], dtype='int64'),
 (Timestamp('2019-09-30 00:00:00'), 'M76'): Int64Index([8], dtype='int64'),
 (Timestamp('2019-09-30 00:00:00'), 'polaris'): Int64Index([9], dtype='int64'),
 (Timestamp('2019-10-01 00:00:00'), 'M27'): Int64Index([10], dtype='int64'),
 (Timestamp('2019-10-01 00:00:00'), 'M33'): Int64Index([11], dtype='int64'),
 (Timestamp('2019-10-01 00:00:00'), 'M57'): Int64Index([12], dtype='int64'),
 

In [22]:
import datetime

In [23]:
def linksInfo(obj,typev):
    if obj == "moon":
        return "https://en.wikipedia.org/wiki/Moon"
    elif typev == "deep":
        return "https://en.wikipedia.org/wiki/Messier_%s"% obj.replace("M","")
    else :
        return "https://en.wikipedia.org/wiki/%s"%obj
    



In [24]:
c = Calendar()
for name,g in cgb:
    #print(name,g)
    night,obj = name

    typev = g["typev"].values[0]
    
    
    e = Event()
    e.name = "%s"%obj
    
    e.begin = str(g["amin"].values[0])
    e.end = str(g["amax"].values[0])
    e.description = """%s Visible during %s Minutes

%s


%s %s  -> %s %s 
    
    
    """%(obj,int(g["dur"].values[0]/60.), linksInfo( obj, typev),
                                            g["alt_end"].values[0],g["az_end"].values[0],
                                            g["alt_start"].values[0],g["az_start"].values[0])
    
    #print(e.description)
    e.categories = ("scope", typev )
    c.events.add(e)


In [25]:
# [<Event 'My cool event' begin:2014-01-01 00:00:00 end:2014-01-01 00:00:01>]
with open('./calendar.ics', 'w') as my_file:
    my_file.writelines(c)